In [1]:
import numpy as np 
import keras
import matplotlib.pyplot as plt
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPool2D ,MaxPooling2D, Dropout
from keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
from keras.utils import to_categorical

Using TensorFlow backend.


In [0]:
train_X = np.load('/content/drive/My Drive/clothing_pred/train_photos.npy')
train_Y = np.load('/content/drive/My Drive/clothing_pred/train_labels.npy')
test_X = np.load('/content/drive/My Drive/clothing_pred/test_photos.npy')
test_Y = np.load('/content/drive/My Drive/clothing_pred/test_labels.npy')

In [0]:
train_Y = to_categorical(train_Y)
train_Y.shape
train_y= train_Y[:,1:]
test_Y = to_categorical(test_Y)
test_Y.shape
test_y= test_Y[:,1:]

In [0]:
input_shape=(227,227,3)
import tensorflow as tf

In [0]:
model = Sequential()
model.add(Conv2D(filters=96, input_shape=(227,227,3), kernel_size=(11,11), strides=(4,4), padding="valid", activation = "relu"))
model.add(MaxPool2D(pool_size=(3,3), strides=(2,2), padding="valid"))
model.add(Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), padding="valid", activation = "relu"))
model.add(MaxPool2D(pool_size=(3,3), strides=(2,2), padding="valid"))
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding="valid", activation = "relu"))
model.add(MaxPool2D(pool_size=(3,3), strides=(2,2), padding="valid"))
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding="valid", activation = "relu"))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(units = 9216, activation = "relu"))
model.add(Dense(5, activation = "softmax"))

In [8]:
from keras.callbacks import EarlyStopping

model.compile(loss='categorical_crossentropy',
                  optimizer=Adam(lr=0.00001),
                  metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 55, 55, 96)        34944     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 23, 23, 256)       614656    
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 11, 11, 256)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 9, 9, 384)         885120    
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 4, 4, 384)         0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 2, 2, 384)        

In [9]:
model.fit(train_X, train_y,
              shuffle=True,
              epochs=25,
              callbacks=[EarlyStopping(min_delta=0.001, patience=3)])

Epoch 1/25
7000/7000 [==============================] - 495s 71ms/step - loss: 1.7155 - accuracy: 0.5383
Epoch 2/25


/usr/local/lib/python3.6/dist-packages/keras/callbacks/callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


7000/7000 [==============================] - 492s 70ms/step - loss: 0.6994 - accuracy: 0.7443
Epoch 3/25
7000/7000 [==============================] - 495s 71ms/step - loss: 0.5352 - accuracy: 0.8096
Epoch 4/25
7000/7000 [==============================] - 494s 71ms/step - loss: 0.4395 - accuracy: 0.8431
Epoch 5/25
7000/7000 [==============================] - 493s 70ms/step - loss: 0.3749 - accuracy: 0.8704
Epoch 6/25
7000/7000 [==============================] - 496s 71ms/step - loss: 0.3198 - accuracy: 0.8877
Epoch 7/25
7000/7000 [==============================] - 497s 71ms/step - loss: 0.3023 - accuracy: 0.8946
Epoch 8/25
7000/7000 [==============================] - 496s 71ms/step - loss: 0.2613 - accuracy: 0.9097
Epoch 9/25
7000/7000 [==============================] - 497s 71ms/step - loss: 0.2252 - accuracy: 0.9214
Epoch 10/25
7000/7000 [==============================] - 497s 71ms/step - loss: 0.2096 - accuracy: 0.9289
Epoch 11/25
7000/7000 [==============================] - 496s 71m

In [10]:
scores = model.evaluate(test_X,test_y)

500/500 [==============================] - 9s 18ms/step


In [11]:
    print('Loss: %.3f' % scores[0])
    print('Accuracy: %.3f' % scores[1])

Loss: 0.147
Accuracy: 0.954


In [0]:
from keras.models import load_model
model.save('my_model2.h5')

In [0]:
from keras.models import load_model
model2 = load_model('my_model2.h5')

In [0]:
predic =model.predict(test_X)

In [15]:
predic.shape

(500, 5)

In [21]:
predic[143]

array([1.5726427e-03, 9.9831164e-01, 7.7383891e-05, 7.6445122e-06,
       3.0661329e-05], dtype=float32)

In [0]:
y_pred = (predic > 0.8)

In [0]:
from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(test_y.argmax(axis=1), y_pred.argmax(axis=1))

In [24]:
matrix

array([[ 94,   0,   2,   0,   4],
       [  3,  97,   0,   0,   0],
       [  0,   0, 100,   0,   0],
       [  5,   0,   1,  94,   0],
       [  4,   0,   2,   2,  92]])